In [4]:
import torch
import torch.nn as nn

lf = nn.MSELoss(reduction='none')
device = 'mps'

model = nn.Sequential(
    nn.Conv1d(3, 3, 1),
)

model = model.to(device)

x = torch.randn(128, 10, 3).to(device)
x = x.permute(0, 2, 1)
y = model(x)
y = y.permute(0, 2, 1)[:,:5,:]
y_hat = torch.randn(128, 5, 3).to(device)

loss = lf(y, y_hat)

if (loss < 0).any():
    print('negative')

# no negative report

negative


In [5]:
loss

/Users/nash/Project/fedn/lib/python3.9/site-packages/torch/_tensor_str.py:115: UserWarning: The operator 'aten::nonzero' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  nonzero_finite_vals = torch.masked_select(


tensor([[[-1.0478, -0.7228,  0.5184],
         [-1.1585,  1.3420,  0.1302],
         [-0.6164,  0.0181, -2.0631],
         [ 1.4987, -0.0985,  1.8138],
         [ 1.2271, -2.3635, -0.2508]],

        [[ 0.8011,  0.3035,  0.6035],
         [ 0.4677, -0.5204,  1.0937],
         [-1.3994, -1.1199, -0.6621],
         [ 0.7400, -0.1411, -0.6150],
         [-0.3509, -3.2230, -2.1333]],

        [[-0.1919,  0.0485,  0.1039],
         [-0.1907,  1.3807,  0.7500],
         [ 0.3267,  2.7466, -0.5851],
         [ 1.7844, -0.5899,  0.2378],
         [-0.3245, -2.0354,  0.3644]],

        ...,

        [[ 0.9903, -0.3085,  1.1084],
         [-0.7344,  0.6037,  1.6245],
         [-0.0639,  1.6123,  0.1382],
         [-1.5711,  0.0505, -0.2646],
         [ 1.3256,  1.4281, -0.7906]],

        [[-0.0262,  0.1073, -0.1354],
         [ 0.9377, -0.8808,  1.2319],
         [-0.2413,  0.1202,  0.7507],
         [ 0.9632, -0.5862,  0.0912],
         [ 0.0207,  0.2398,  2.5932]],

        [[-1.3479,  0.2797